<a href="https://colab.research.google.com/github/idontcalculate/kaggletweets/blob/main/KaggleToxicTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aim is to classificate tweets from Kaggle dataset in 6 categories (labels) and to make functional network that can sort tweets and make prediction with test dataset. 

# New Section

# New Section

In [ ]:
#check for GPU
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-55722894-32f6-1db4-f131-27e1d7116c67)


In [28]:
import numpy as np

In [ ]:
!pip install kaggle 

In [ ]:
!mv /content/kaggle.json /root/.kaggle

mv: cannot stat '/content/kaggle.json': No such file or directory


In [ ]:
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_labels.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-06-08 12:55:21--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-06-08 12:55:21 (90.8 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [1]:
!nvidia-smi

Tue Jun  8 13:16:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [237]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [ ]:
!unzip /content/test_labels.csv.zip

Archive:  /content/test_labels.csv.zip
replace test_labels.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!unzip /content/train.csv.zip

In [ ]:
!unzip /content/test.csv.zip

In [82]:
import pandas as pd 
train_df = pd.read_csv('/content/train.csv.zip')
test_df  = pd.read_csv('/content/test.csv.zip')

In [83]:
test_labels = pd.read_csv('/content/test_labels.csv.zip')

In [84]:
len(train_df), len(test_df)

(159571, 153164)

In [85]:
# How many samples total?
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 159571
Total test samples: 153164
Total samples: 312735


In [140]:
def create_text_label(dataframe):
  texts = dataframe.comment_text.to_numpy()
  labels = dataframe.drop(columns =['id','comment_text']).to_numpy()
  return texts,labels


In [141]:
x_train,y_train = create_text_label(train_df)

In [142]:
x_test,y_test = create_text_label(test_df)

In [144]:
y_test = test_labels.drop('id',axis = 1).to_numpy()

In [145]:
x_train[:10],y_train[:10]

(array(["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
        "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
        "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",
        '"\nMore\nI can\'t make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferenc

In [146]:
x_test[:10],y_test[:10]

(array(["Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,",
        '== From RfC == \n\n The title is fine as it is, IMO.',
        '" \n\n == Sources == \n\n * Zawe Ashton on Lapland —  /  "',
        ":If you have a look back at the source, the information I updated was the correct form. I can only guess the source hadn't updated. I shall update the information once again but thank you for your message.",
        "I don't anonymously edit articles at all.",
        'Thank you for understanding. I think very highly of you and would not revert without discussion.',
        'Please do not add nonsense to Wikipedia. Such edits are considered vandalism and quickly undone. If 

In [136]:
y_train.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


In [62]:
len(x_train)

159571

In [63]:
len(train_data_text[0])

264

In [147]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long should the output sequence of tokens be?
                                    pad_to_max_tokens=True)

For max_tokens (the number of words in the vocabulary), multiples of 10,000 (10,000, 20,000, 30,000) or the exact number of unique words in your text (e.g. 32,179) are common values.

For our use case, we'll use 10,000.

And for the output_sequence_length we'll use the average number of tokens per Tweet in the training set. But first, we'll need to find it.

In [148]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

putting dataframe into data_sentences, and then making and array, before vectorizing. 

In [149]:
text_vectorizer.adapt(x_train)
# x_test = text_vectorizer.adapt(np.array(x_test))


text_vectorizer.adapt(y_train)

In [150]:
text_vectorizer(x_train[:1])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 645,   76,    2,  122,  127,  172,   29,  636, 4604,    1, 1282,
          83,  313,   53, 2076]])>

In [151]:
sample_scentence = 'this is the sample scentence'
text_vectorizer([sample_scentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  14,    9,    2, 4705,    1,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [152]:
text_vocab = text_vectorizer.get_vocabulary()

In [154]:
text_vectorizer(x_test)

<tf.Tensor: shape=(153164, 15), dtype=int64, numpy=
array([[2661,  762,    1, ...,   81,   22,    7],
       [  31, 1246,    2, ...,    0,    0,    0],
       [ 107,    1,    1, ...,    0,    0,    0],
       ...,
       [   1, 1212,    8, ...,    8,   99, 2591],
       [  50,    4,    2, ...,    4, 1384,  357],
       [ 168,  226,   21, ...,   23,    7,   69]])>

In [155]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'to', 'of']
Bottom 5 least common words: ['domination', 'divorced', 'distorting', 'direktor', 'dildo']


activating embedding layer

In [156]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length) # how long is each input

embedding

In [162]:
# Get a random sentence from training set
import random
random_sentence = random.choice(x_train)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
"

 hi 

yes I was in the process of doing that.  One thing, however.  Are you familiar with the distinction between ""pages for deletion"" and ""votes for deletion?""  What is going on there? "      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03662921,  0.04177663,  0.034984  , ...,  0.01993462,
         -0.01815165, -0.0136642 ],
        [-0.00978785,  0.02481626, -0.01908884, ...,  0.01024635,
          0.03374508, -0.02607433],
        [-0.00246595, -0.00769802, -0.04085737, ...,  0.00209464,
          0.04397613,  0.02414354],
        ...,
        [ 0.03883507,  0.01510395, -0.03064976, ...,  0.02610817,
          0.0307348 ,  0.02778542],
        [ 0.04470099,  0.01998186,  0.01321849, ..., -0.00013406,
         -0.02289292, -0.00056386],
        [ 0.04080817, -0.00557964,  0.0499302 , ...,  0.01024337,
         -0.00065855,  0.03001625]]], dtype=float32)>

In [163]:
#single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 3.66292112e-02,  4.17766310e-02,  3.49840038e-02, -9.54092667e-03,
       -2.20099576e-02,  5.31911850e-04,  5.92816621e-04, -4.17434089e-02,
       -7.10426643e-03, -3.06384694e-02, -5.44846058e-03,  2.18055397e-03,
       -1.42780542e-02, -2.50807405e-02,  3.74308117e-02,  3.32780741e-02,
        2.45540254e-02, -1.90954208e-02,  1.25467218e-02,  4.64549400e-02,
        4.48097251e-02, -3.12320720e-02, -5.04038483e-03,  1.83659457e-02,
       -3.25421244e-02, -4.24530506e-02,  4.98774983e-02, -7.39242882e-03,
       -4.31500338e-02, -4.92305420e-02,  1.94595493e-02,  5.03449515e-03,
        4.94187139e-02, -2.03878041e-02,  4.78857197e-02, -3.11423671e-02,
       -7.19553232e-03, -9.22361761e-03,  4.84148748e-02,  1.89566351e-02,
       -1.99457537e-02, -4.05566767e-03, -7.99406320e-04,  1.53911598e-02,
       -2.11711656e-02, -3.50661166e-02,  6.52503967e-03, -1.40803829e-02,
       -2.25584861e-02, -3.46636176e-02,  4.11222838

creating the basseline model

In [168]:
x_train.shape, y_train.shape

((159571,), (159571, 6))

In [200]:
#SAVE DIR AND CALLBACKS

from helper_functions import create_tensorboard_callback

#create dir to save tensorboard logs
SAVE_DIR = "model_logs"

#print(type(SAVE_DIR))

In [201]:
# Create LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = layers.LSTM(64)(x) # return vector for whole sequence
print(x.shape)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(6, activation="softmax")(x)
model_1 = tf.keras.Model(inputs, outputs, name="LTSM_kaggle")

(None, 15, 128)
(None, 64)


In [202]:
# Compile model
model_1.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [203]:
# F model
model_1_history = model_1.fit(x_train,
                              y_train,
                              epochs=5,
                              validation_data=(x_test, y_test),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "LSTM_kaggle")])

Saving TensorBoard log files to: model_logs/LTSM_kaggle/20210608-170743
Epoch 1/5
4987/4987 [==============================] - 111s 22ms/step - loss: 0.3785 - accuracy: 0.9377 - val_loss: -6.5668 - val_accuracy: 0.8452
Epoch 2/5
4987/4987 [==============================] - 104s 21ms/step - loss: 0.3619 - accuracy: 0.9346 - val_loss: -6.5858 - val_accuracy: 0.8366
Epoch 3/5
4987/4987 [==============================] - 104s 21ms/step - loss: 0.3490 - accuracy: 0.9336 - val_loss: -6.6129 - val_accuracy: 0.8241
Epoch 4/5
4987/4987 [==============================] - 106s 21ms/step - loss: 0.3390 - accuracy: 0.9331 - val_loss: -6.6006 - val_accuracy: 0.8313
Epoch 5/5
4987/4987 [==============================] - 104s 21ms/step - loss: 0.3322 - accuracy: 0.9352 - val_loss: -6.6302 - val_accuracy: 0.8543


In [204]:
!pip install tensorboard

In [212]:
!tensorboard dev upload --logdir ./model_logs/LTSM_kaggle --one_shot

2021-06-08 17:42:16.276228: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Data for the "text" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/1IOglpC4Sky3KEvH3nuBSA/

[2021-06-08T17:42:17] Started scanning logdir.
[2021-06-08T17:42:18] Total uploaded: 30 scalars, 0 tensors, 1 binary objects (255.8 kB)
[2021-06-08T17:42:18] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/1IOglpC4Sky3KEvH3nuBSA/


In [211]:
SAVE_DIR

'model_logs'

In [215]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long should the output sequence of tokens be?
                                    pad_to_max_tokens=True)

In [216]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length,
                                    ngrams=2)

In [217]:
#crate model 2 for kaggle

# Create another LTSM 
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = layers.LSTM(64)(x) # return vector for whole sequence
print(x.shape)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(6, activation="softmax")(x)
model_2 = tf.keras.Model(inputs, outputs, name="LTSM_kaggle_2")


(None, 15, 128)
(None, 64)


In [218]:
# Compile model
model_2.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [229]:
#fit the model
model_2_history = model_2.fit(x_train,
                              y_train,
                              epochs=10,
                              validation_data=(x_test, y_test),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       "LTSM_kaggle_2")])

Saving TensorBoard log files to: model_logs/LTSM_kaggle_2/20210608-182429
Epoch 1/10
4987/4987 [==============================] - 114s 23ms/step - loss: 0.3360 - accuracy: 0.9921 - val_loss: -6.4351 - val_accuracy: 0.9990
Epoch 2/10
4987/4987 [==============================] - 111s 22ms/step - loss: 0.3305 - accuracy: 0.9942 - val_loss: -6.5016 - val_accuracy: 0.9990
Epoch 3/10
4987/4987 [==============================] - 112s 22ms/step - loss: 0.3268 - accuracy: 0.9942 - val_loss: -6.5434 - val_accuracy: 0.9990
Epoch 4/10
4987/4987 [==============================] - 131s 26ms/step - loss: 0.3240 - accuracy: 0.9942 - val_loss: -6.6141 - val_accuracy: 0.9990
Epoch 5/10
4987/4987 [==============================] - 112s 22ms/step - loss: 0.3218 - accuracy: 0.9942 - val_loss: -6.6888 - val_accuracy: 0.9990
Epoch 6/10
4987/4987 [==============================] - 131s 26ms/step - loss: 0.3200 - accuracy: 0.9942 - val_loss: -6.7121 - val_accuracy: 0.9990
Epoch 7/10
4987/4987 [================

In [231]:
#check the results
model_2.evaluate(x_test, y_test)

4787/4787 [==============================] - 25s 5ms/step - loss: -6.8439 - accuracy: 0.9990


[-6.843910217285156, 0.9990010857582092]

In [232]:
#predictions
model_2_pred_probs = model_2.predict(x_test)
model_2_pred_probs.shape

(153164, 6)

In [233]:
model_2_pred_probs[:10]

array([[0.32410717, 0.09550231, 0.23191188, 0.04867075, 0.21802068,
        0.08178723],
       [0.32410717, 0.09550231, 0.23191188, 0.04867075, 0.21802068,
        0.08178723],
       [0.32410717, 0.09550231, 0.23191188, 0.04867075, 0.21802068,
        0.08178723],
       [0.32410717, 0.09550231, 0.23191188, 0.04867075, 0.21802068,
        0.08178723],
       [0.32410717, 0.09550231, 0.23191188, 0.04867075, 0.21802068,
        0.08178723],
       [0.32410717, 0.09550231, 0.23191188, 0.04867075, 0.21802068,
        0.08178723],
       [0.32410717, 0.09550231, 0.23191188, 0.04867075, 0.21802068,
        0.08178723],
       [0.32410717, 0.09550231, 0.23191188, 0.04867075, 0.21802068,
        0.08178723],
       [0.32410717, 0.09550231, 0.23191188, 0.04867075, 0.21802068,
        0.08178723],
       [0.32410717, 0.09550231, 0.23191188, 0.04867075, 0.21802068,
        0.08178723]], dtype=float32)

In [235]:
# Turn prediction probabilities into single-dimension tensor of floats
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs)) # squeeze removes single dimensions
model_2_preds[:50]

<tf.Tensor: shape=(50, 6), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.,

results should be in panda dataframe in order to submit. 